In [40]:
import urllib.request # for downloading wordle words from source
import numpy as np # for stats
import random # for randomly generating target and start words
import operator # for sorting letter frequency distribution
import time # for #dramaticeffect
# from collections import defaultdict # for letter frequency distribution

## Making Datasets
- Get all possible words that the target word could be
- For each word in the target words list, get counts of each letter to create letter distribution across entire vocabulary

### alt_words_1
- len() == 14855

In [41]:
### If getting words from local file -- should be 14855 words in total

alt_words_1 = set() # set of all words

file_path = "data/alt_words_1.txt" # taken from "https://raw.githubusercontent.com/tabatkins/wordle-list/main/words"
f = open(file_path, "r", encoding = "utf-8")

with open(file_path, "r", encoding = "utf-8") as f:
    for word in f.read().split("\n"):
        if len(word) > 0: # there's one blank entry at the start
            alt_words_1.add(word)

f.close() # closes connection to file

print(len(alt_words_1))
alt_words_1 = list(alt_words_1)
alt_words_1[:10]

14855


['utero',
 'paxes',
 'soken',
 'grape',
 'seems',
 'cotes',
 'accha',
 'homas',
 'ulans',
 'airth']

### official_words list
- len() == 2309

In [42]:
official_words = set() # set of all words

file_path = "data/official_words.txt"
f = open(file_path, "r", encoding = "utf-8")

with open(file_path, "r", encoding = "utf-8") as f:
    for line in f.read().split("\n"):
        word = line.split(" ")[-1]
        if (len(word) != 5 or word.isalpha() == False):
            pass
        else:
            official_words.add(word.lower())

f.close() # closes connection to file

for word in official_words:
    if len(word) != 5:
        print (word)

official_words = list(set(official_words))
print(len(official_words))
official_words[:10]

2309


['realm',
 'grape',
 'torso',
 'carat',
 'metro',
 'louse',
 'jewel',
 'watch',
 'agate',
 'smell']

In [43]:
### Gets most common letters of all words of the dataset

def get_letter_counts(letters: str, word_list: list, sort: str = "descending"):
    """
    Given a passed str of letters and a list of words, produces a frequency distribution of all letters
    
    ------
    Parameters:
    ------
    `letters`: str
        a string of letters to be counted. String must only be desired letters, with no spaces. Default is local variable containing all letters of the English alphabet
    `word_list`: list
        list of words (str) from which word frequencies will be counted
    `sort`: str
        if either "descending" or "ascending" are passed, returned list of tuples will be sorted accoringly, else returned dictionary will be unsorted

    ------
    Returns:
    ------
    `letters_counts_dict`: dict
        dictionary of {letter : count} pairs for each letter in passed `letters` sequence
    `sorted_counts_dicts`: list of tuples
        list of tuples. Format is ("letter", frequency). Ordered according to `sort` values
    """

    words_counts_dict = {}

    # for word in ten_words: # test dataset
    for word in word_list: # real dataset
        word_dict = {}
 
        for letter in word:
            if letter in word_dict:
                word_dict[letter] += 1
            else:
                word_dict[letter] = 1
        words_counts_dict[word] = word_dict

    letters_counts_dict = {}

    for letter in letters:
        letters_counts_dict[letter] = 0

    for word, count_dict in words_counts_dict.items():
        # print (word, count_dict)
        for letter, count in count_dict.items():
            letters_counts_dict[letter] += count

    if sort == "ascending":
        sorted_counts_dict = (sorted(letters_counts_dict.items(), key = operator.itemgetter(1), reverse = False))
        return sorted_counts_dicts

    if sort == "descending":
        sorted_counts_dict = sorted(letters_counts_dict.items(), key = operator.itemgetter(1), reverse = True)
        return sorted_counts_dict

    else:
        return letters_counts_dict

In [44]:
english_alphabet = "abcdefghijklmnopqrstuvwxyz"

alt_sorted_counts = get_letter_counts(english_alphabet, alt_words_1, sort = "descending")
alt_sorted_counts

[('e', 7455),
 ('s', 7319),
 ('a', 7128),
 ('o', 5212),
 ('r', 4714),
 ('i', 4381),
 ('l', 3780),
 ('t', 3707),
 ('n', 3478),
 ('u', 2927),
 ('d', 2735),
 ('p', 2436),
 ('m', 2414),
 ('y', 2400),
 ('c', 2246),
 ('h', 1993),
 ('g', 1864),
 ('b', 1849),
 ('k', 1753),
 ('f', 1240),
 ('w', 1127),
 ('v', 801),
 ('z', 503),
 ('j', 342),
 ('x', 326),
 ('q', 145)]

In [45]:
english_alphabet = "abcdefghijklmnopqrstuvwxyz"

official_sorted_counts = get_letter_counts(english_alphabet, official_words, sort = "descending")
official_sorted_counts

[('e', 1230),
 ('a', 975),
 ('r', 897),
 ('o', 753),
 ('t', 729),
 ('l', 716),
 ('i', 670),
 ('s', 668),
 ('n', 573),
 ('c', 475),
 ('u', 466),
 ('y', 424),
 ('d', 393),
 ('h', 387),
 ('p', 365),
 ('m', 316),
 ('g', 310),
 ('b', 280),
 ('f', 229),
 ('k', 210),
 ('w', 194),
 ('v', 152),
 ('z', 40),
 ('x', 37),
 ('q', 29),
 ('j', 27)]

### `best_guess_words()` function

In [46]:
### Best first guesses for official wordle list

best_opening_words = []
best_letters = set()

### Get 5 top most frequent letters 
for letter, freq in official_sorted_counts:
    best_letters.add(letter)
    if len(best_letters) == 4:
        break

### Get all words that have one of each of the 5 top most frequent letters
for word in official_words:
    word_set = set()

    for letter in word:
        word_set.add(letter)

    if best_letters.issubset(word_set):
        best_opening_words.append(word)

print(best_letters)
print(best_opening_words)

{'o', 'r', 'e', 'a'}
['adore', 'opera', 'arose']


### `best_guess_words()` function
- given a passed `word_list`, return the best possible words to guess in Wordle

In [136]:
### Best first guesses for a given Wordle list

def best_guess_words(word_list: list, show_letters: bool = False):
    """
    Given a passed list of English words of a consistent length, calculates the most statistically optimal first guess words, alongside a rating for each word. 
    
    Rating = sum(frequency of each unique letter in that word) / sum (all unique letter frequencies in word_list) * 100, rounded to 2 decimals.

    ------
    Parameters:
    ------
    `word_list`: list
        list of words (str) of consistent length
    `show_letters`: bool
        if True, also prints set of most optimal letters to guess

    ------
    Returns:
    ------
    `word_ratings`: list
        list of tuples. Format is [(word, rating)], where rating is calculated according to above formula
    `sorted_counts`: list of tuples
        list of tuples. Format is ("letter", frequency). Sorted according to `sort` value; ["descending" or "ascending"] if passed
    """
        
    english_alphabet = "abcdefghijklmnopqrstuvwxyz"

    sorted_counts = get_letter_counts(english_alphabet, word_list, sort = "descending")

    max_len_possible = len(word_list[0])

    ### Get words with the highest letter diversity
    while max_len_possible > 0: # so worst case scenario, max_length_possible stops at 1

        best_letters = set()
        best_words = []

        for letter, freq in sorted_counts:
            best_letters.add(letter)
            if len(best_letters) == max_len_possible:
                break

            # print (best_letters)

        ### Get all words that have one of each of the 5 top most frequent letters
        for word in word_list:
            word_set = set()

            for letter in word:
                word_set.add(letter)
            # print (word_set)

            if best_letters.issubset(word_set):
                best_words.append(word)

        if len(best_words) > 0:
            break
        else:
            max_len_possible -= 1 # only try the top 4 letters, then 3, then 2, ...
        
        if max_len_possible == 0:
            break

    all_letters_count = 0
    for letter, freq in sorted_counts:
        all_letters_count += freq

    word_ratings = []
    for word in best_words:
        ratings_dict = {}
        for letter in word:
            for freq_letter, freq in sorted_counts:
                if letter == freq_letter:
                    ratings_dict[letter] = freq
        
        total_rating = 0
        for letter, rating in ratings_dict.items():
            total_rating += rating
        
        word_ratings.append((word, round(total_rating / all_letters_count * 100, 2)))

    word_ratings = sorted(word_ratings, key = operator.itemgetter(1), reverse = True)

    if show_letters == True:
        # return best_words, sorted_counts
        return word_ratings, sorted_counts
    else:
        # return best_words
        return word_ratings

In [139]:
# best_guess_words(alt_words_1, show_letters = True)
best_guess_words(official_words, show_letters = False)

[('arose', 39.18), ('adore', 36.8), ('opera', 36.55)]

## Solving Wordle Functions

### `solve_wordle()` function versions -- these DON'T work

In [47]:
# ### VERSION 1

# def solve_wordle(opening_guess: str, target_word: str, guesses = 0, verbose = False):
#     """
#     Mimicking the popular web game, this function matches a current word to a target word in the most statistically optimal way possible.

#     -----
#     Parameters:
#     opening_guess: str
#         a five-letter string
#     target_word: str
#         a five-letter string
#     verbose: bool
#         if True, prints progress and explanation of how function solves the puzzle. If False, prints only the guessed word at each guess.

#     -----
#     Returns:
#         None
#     """

#     all_letter_positions = set(i for i in range(0, 5)) # creates a set of 0-4 (for each letter position) -- just to avoid regenerating this each time it's needed
#     next_guess_list = ['_' for i in range(0, 5)] # initializes new list of 5 "_"

#     correct_positions = set()

#     incorrect_positions_words = set()
#     incorrect_letters = set()

#     incorrect_pos_letters = set()
#     incorrect_positions = set()

#     if guesses == 0:

#         if verbose == True:
#             print(f"----------------------------------------------------\n")

#     guesses += 1

#     if verbose == True:
#         print (f"Guess {guesses}:\n\t{opening_guess}\n")
#     else:
#         print (f"Guess {guesses}: {opening_guess}")

#     # evaluating current guess against target and returning correct letters
#     for i in all_letter_positions: # 5 letters in each word (current word and target word)
#         corr_curr_letts_list = ['_' for i in range(0, 5)]
#         incorr_curr_letts_list = ['_' for i in range(0, 5)]
#         incorr_pos_letters_list = ['_' for i in range(0, 5)]
        
#         if opening_guess[i] == target_word[i]: # if letters exist and are in the correct position
#             next_guess_list[i] = opening_guess[i]
#             correct_positions.add(i)            

#         elif opening_guess[i] not in target_word: # if letters do not exist at all in target word
#             incorrect_letters.add(opening_guess[i])
#             incorrect_positions_words.add(i)

#         elif (opening_guess[i] != target_word[i] and  opening_guess[i] in target_word): # if not at that position but is elsewhere in the word
#             incorrect_pos_letters.add(opening_guess[i])
#             incorrect_positions.add(i)

#     if verbose == True:

#         if len(correct_positions) > 0:
#             for pos in correct_positions:
#                 corr_curr_letts_list[pos] = opening_guess[pos]
#                 corr_curr_letts_str = " ".join(corr_curr_letts_list)

#             print (f"Current letters in correct locations:\n\t{corr_curr_letts_str}\n")
        
#         if len(incorrect_positions) > 0:
#             for pos in incorrect_positions:
#                 incorr_pos_letters_list[pos] = opening_guess[pos]
#                 incorr_pos_letts_str = " ".join(incorr_pos_letters_list)

#             print (f"Current correct letters in incorrect locations:\n\t{incorr_pos_letts_str}\n")

#         if len(incorrect_positions_words) > 0:
#             for pos in incorrect_positions_words:
#                 incorr_curr_letts_list[pos] = opening_guess[pos]
#                 incorr_curr_letts_str = " ".join(incorr_curr_letts_list)

#             print (f"Current letters not in target word:\n\t{incorr_curr_letts_str}\n")
#         else:
#             print (f"All current letters are in target word.\n")


#     # if the guess match the target, print a successful response and end function call
#     if opening_guess == target_word:

#         if guesses == 1:
#             print (f"Wordle has been solved in {guesses} guess!")
        
#         else:
#             print (f"Wordle has been solved in {guesses} guesses!")
#             print (f"{opening_guess}")
    
#     # if guess does not match the target, continue calling function until either max guesses is reached or guessed word matches target
#     else:
#         incorrect_positions_letters = all_letter_positions.difference(correct_positions) # works fine

#         # print (incorrect_letters, next_guess_list)

#         if len(incorrect_pos_letters) > 0:
            
#             for new_guess_position in incorrect_positions_letters:
#                 # print(new_guess_position)

#                 for incorr_pos_letter in list(incorrect_pos_letters):

#                     next_guess_list[new_guess_position] = incorr_pos_letter
            
#                     if "_" not in next_guess_list:
#                         next_guess_str = "".join(next_guess_list)
                        
#                         if next_guess_str in all_words:
#                             break

#         # fill empty spots with next best new unused letters
#         else:
            
#             for letter, freq in sorted_letters_counts:

#                 if letter not in (incorrect_letters or next_guess_list):
                    
#                     for new_guess_position in incorrect_positions_letters:
#                         next_guess_list[new_guess_position] = letter

#                 if "_" not in next_guess_list:
#                     next_guess_str = "".join(next_guess_list)
                    
#                     if next_guess_str in all_words:
#                         break

#         if verbose == True:
#             print(f"----------------------------------------------------\n")

#         # recursively call function with new guess until max number of tries is reached
#         if guesses < len(opening_guess):
#             opening_guess = next_guess_str
#             solve_wordle(opening_guess, target_word, guesses, verbose)

#         else:
#             print(f"\nUnfortunately, this Wordle could not be solved in {guesses} guesses.\n\nThe correct word was '{target_word}'.\n\nBetter luck next time.")

#             if verbose == True:
#                 print(f"\n----------------------------------------------------")

# # solve_wordle(opening_word, target_word, verbose = True)
# solve_wordle("fried", target_word, verbose = True)

In [48]:
# ### VERSION 2

# def solve_wordle(opening_guess: str, target_word: str, guesses = 0, verbose = False):
#     """
#     Mimicking the popular web game, this function matches a current word to a target word in the most statistically optimal way possible.

#     -----
#     Parameters:
#     opening_guess: str
#         a five-letter string
#     target_word: str
#         a five-letter string
#     verbose: bool
#         if True, prints progress and explanation of how function solves the puzzle. If False, prints only the guessed word at each guess.

#     -----
#     Returns:
#         None
#     """

#     all_letter_positions = set(i for i in range(0, 5)) # creates a set of 0-4 (for each letter position) -- just to avoid regenerating this each time it's needed

#     if guesses == 0:

#         if verbose == True:
#             print(f"----------------------------------------------------\n")

#     guesses += 1

#     if verbose == True:
#         print (f"Guess {guesses}:\n\t{opening_guess}\n")
#     else:
#         print (f"Guess {guesses}: {opening_guess}")

#     corr_all_dict = {} # letter == correct and position == correct
#     incorr_letts_set = set() # can be set because position doesn't matter. Letter is not relevant at all
#     corr_letts_incorr_pos_dict = {} # letter == correct and position != correct

#     # evaluating current guess against target and returning correct letters
#     for i in all_letter_positions: # 5 letters in each word (current word and target word)
#         corr_all_dict[opening_guess[i]] = set()
#         corr_letts_incorr_pos_dict[opening_guess[i]] = set()
        
#         if opening_guess[i] == target_word[i]: # letter == correct and position == correct
#             corr_all_dict[opening_guess[i]].add(i)

#         elif opening_guess[i] not in target_word: # if letter is not relevant at all
#             incorr_letts_set.add(opening_guess[i])

#         elif (opening_guess[i] != target_word[i] and  opening_guess[i] in target_word): # letter == correct and position != correct
#             corr_letts_incorr_pos_dict[opening_guess[i]].add(i)

#     print (f"corr_all_dict: {corr_all_dict}")
#     print(f"incorr_letts_set: {incorr_letts_set}")
#     print(f"corr_letts_incorr_pos_dict: {corr_letts_incorr_pos_dict}")

#     next_guess_list = ["_" for i in range(0, 5)] # initializes new list of 5 "_"

#     # filling next guess with current correct letters in correct locations
#     for lett, positions in corr_all_dict.items():
#         if len(positions) > 0:
#             for pos in positions:
#                 next_guess_list[pos] = lett
    
#     # getting locations of all empty positions still in list
#     if "_" in next_guess_list:
        
#         # get empty positions(s)
#         empty_positions = set()
#         for i in next_guess_list:
#             if i == "_":
#                 empty_positions.add(next_guess_list.index(i))

#         # using current letter(s) in incorrect locations to fill empty position(s)
#         for lett, positions in corr_letts_incorr_pos_dict.items():
#             if len(positions) > 0:
#                 for pos in empty_positions.difference(positions):
#                     next_guess_list[pos] = lett

#         # get empty positions again
#         empty_positions = set()
#         for i in next_guess_list:
#             if i == "_":
#                 empty_positions.add(next_guess_list.index(i))                    

#         # using new letter(s) in incorrect locations to fill empty position(s)

#         for i in empty_positions:
#             for lett, freq in sorted_letters_counts:
#                 if lett not in (incorr_letts_set or next_guess_list):
#                     next_guess_list[i] = lett

#                     # check if new word is in wordle list:
#                     next_guess_str = "".join(next_guess_list)
#                     if next_guess_str in all_words:
#                         break

#         print(next_guess_str)       
#         # solve_wordle(next_guess_str, target_word, guesses, verbose)
#         print ("still blanks")
#     else:
#         print ("no more blanks")
#         next_guess_str = "".join(next_guess_list)

#         # recursively call function until max attempts is reached or wordle is solved
#         # solve_wordle(next_guess_str, target_word, guesses, verbose)

# # solve_wordle(opening_word, target_word, verbose = True)
# solve_wordle("aeros", target_word, verbose = False)

In [49]:
# ### VERSION 3

# def solve_wordle(opening_word: str, target_word: str, max_attempts: int = len(opening_word), verbose: bool = False, drama: float = None):
#     """
#     Mimicking the popular web game, this function matches a current word to a target word in the most statistically optimal way possible.

#     ------
#     Parameters:
#     ------
#     `opening_word`: str
#         a string -- must be the same length as `target_word`
#     `target_word`: str
#         a string -- must be the same length as `opening_word`
#     `max_attempts`: int
#         the maximum number of attempts allowed to solve the Wordle
#     `verbose`: bool
#         if True, prints progress and explanation of how function solves the puzzle. If False, prints only the guessed word at each guess.
#     `drama`: float or int
#         if int provided, each guess' output is delayed by that number of seconds, else each output is shown as quickly as possible. For ~dRaMaTiC eFfEcT~

#     ------
#     Returns:
#     ------
#         None
#     """

#     incorr_letts_set = set() # can be a set because position doesn't matter (only considers unique letters)
#     all_letter_positions = set(i for i in range(0, len(opening_word)))
#     guesses = 0
#     opening_word_list = [letter for letter in opening_word]

#     ### Can set a number of max guesses different than 5
#     if max_attempts:
#         max_guesses = max_attempts
#     else:
#         max_guesses = len(opening_word)

#     if opening_word == target_word:
#         print(f"Congratulations! The Wordle was solved on the first try. That's incredible!")
#         if max_attempts == 2:
#             print(f"There was 1 guess remaining.")
#         else:
#             print(f"There were still {max_attempts - 1} guesses remaining.")
#         # print(f"Here are some statistics about this Wordle:") # come up with some stats about it
#         print("\n-----------------------------")

#     ### "GUESSING" BEGINS
#     # while opening_word != target_word:
#     while opening_word:

#         if drama:
#             if type(drama) == float or int:
#                 time.sleep(np.absolute(drama))

#         corr_all_dict = {} # letter == correct and position == correct
#         corr_letts_incorr_pos_dict = {} # letter == correct and position != correct

#         if guesses == 0:
#             if verbose == True:
#                 print("-----------------------------\n")
        
#         guesses += 1
        
#         if verbose == True:
#             print(f"Guess {guesses}:\n\t{' '.join(opening_word_list)}\n")
#         else:
#             print(f"Guess {guesses}: {opening_word}")

#         ### EVALUATING CURRENT GUESS
#         for i in all_letter_positions: # number of letters in each word (current word and target word)
#             corr_all_dict[opening_word[i]] = set()
#             corr_letts_incorr_pos_dict[opening_word[i]] = set()
            
#             if opening_word[i] == target_word[i]: # letter == correct and position == correct
#                 corr_all_dict[opening_word[i]].add(i)

#             elif (opening_word[i] != target_word[i] and  opening_word[i] in target_word): # letter == correct and position != correct
#                 corr_letts_incorr_pos_dict[opening_word[i]].add(i)

#             elif opening_word[i] not in target_word: # if letter is not relevant at all
#                 incorr_letts_set.add(opening_word[i])

#         ### CREATING NEXT WORD GUESS
#         next_guess_list = ["_" for i in range(0, 5)] # initializes new list of 5 "_"

#         # filling next guess with current correct letters in correct locations
#         for lett, positions in corr_all_dict.items():
#             if len(positions) > 0:
#                 for pos in positions:
#                     next_guess_list[pos] = lett

#         # print (next_guess_list)

#         # getting locations of all empty positions still in list
#         if "_" in next_guess_list:

#             # print ("EMPTIESSS")

#             # get empty positions(s)
#             empty_positions = set()

#             for i, val in enumerate(next_guess_list):

#                 if val == "_":
#                     empty_positions.add(i)

#             # using current letter(s) in incorrect locations to fill empty position(s)
#             for lett, positions in corr_letts_incorr_pos_dict.items():
#                 if len(positions) > 0:
#                     # print (empty_positions.difference(positions))
#                     for pos in empty_positions.difference(positions):
#                         # print (lett)
#                         next_guess_list[pos] = lett

#             # print (incorr_letts_set)
#             # print (next_guess_list)


# ############################# ISSUE IS HERE #############################

#             # using new letter(s) in incorrect locations to fill empty position(s)
#             for i in empty_positions:
#                 for lett, freq in sorted_letters_counts:

#                     # make sure the new letter is one that hasn't been used it
#                     if lett not in (incorr_letts_set or set(next_guess_list)):
#                         next_guess_list[i] = lett

#                     # check if new word is in wordle list:
#                     if "".join(next_guess_list) in all_words:
#                         break
#                     else:
#                         continue

#                 # print (next_guess_list)
#             opening_word = "".join(next_guess_list)

#             # print (opening_word)

# ############################# ISSUE IS HERE #############################

#         if verbose == True:
#             correct_positions = ["_" for i in range(0, len(opening_word))]
#             for lett, positions in corr_all_dict.items():
#                 if len(positions) > 0:
#                     for pos in positions:
#                         correct_positions[pos] = lett

#             incorrect_positions = ["_" for i in range(0, len(opening_word))]
#             for lett, positions in corr_letts_incorr_pos_dict.items():
#                 if len(positions) > 0:
#                     for pos in positions:
#                         incorrect_positions[pos] = lett

#         if opening_word == target_word:
#             if guesses == 1:
#                 print(f"Congratulations! The Wordle has been solved in {guesses} guess!")
#             else:
#                 print(f"\nCongratulations! The Wordle has been solved in {guesses} guesses!")
#                 print(f"There were still {max_attempts - guesses} guesses remaining.")
#                 # print(f"Here are some statistics about this Wordle:") # come up with some stats about it
#                 # print("\n-----------------------------")
#             break
#         else:
#             if verbose == True:
#                 print (f"Correct letters in correct positions:\n\t{' '.join(correct_positions)}\n")
#                 print(f"Correct letters in incorrect positions:\n\t{' '.join(incorrect_positions)}\n")
#                 print(f"All incorrect letters:\n\t{', '.join(letter for letter in incorr_letts_set)}\n")

#         # condition to break the loop
#         if guesses == max_guesses:
#             if verbose == True:
#                 print("-----------------------------\n")
#                 print(f"Unfortunately, the Wordle could not be solved in {max_attempts} guesses.\n")
#                 print(f"The target word was '{target_word}'. Better luck next time!\n")
#                 print("-----------------------------\n")
#             else:
#                 print(f"\nUnfortunately, the Wordle could not be solved in {max_attempts} guesses.")
#                 print(f"The target word was '{target_word}'. Better luck next time!\n")
#             break
        
#         opening_word_list = [letter for letter in opening_word]

#         if verbose == True:
#             print (f"Next guess:\n\t{' '.join(opening_word_list)}\n")
#             print("-----------------------------\n")

# solve_wordle("perks", "agros", max_attempts = 5, verbose = False, drama = 0)

In [50]:
# ### VERSION 4

# guessword = "fired"
# # guessword = "album"
# targetword = "tried"

# guessword = guessword.lower()
# targetword = targetword.lower()

# print (f"Guess: {guessword}")
# print (f"Target: {targetword}\n")

# wordlength = len(guessword)

# simple_solve = True

# letter_positions = set(i for i in range(0, wordlength))
# all_perfect_dict = {}
# wrong_pos_dict = {}
# dont_guess_again_set = set()

# ########################################## THIS WORKS ##########################################            

# next_guess_list = ["_" for i in range(0, wordlength)] # have to establish this ahead of the immediate next section

# ##### FIRST GUESS IS EVALUTED
# for i in letter_positions: # number of letters in each word (current word and target word)
    
#     all_perfect_dict[guessword[i]] = set()
#     wrong_pos_dict[guessword[i]] = set()
#     # dont_guess_again_set[guessword[i]] = set()
    
#     if guessword[i] == targetword[i]: # letter is correct and position is correct
#         # all_perfect_dict[guessword[i]].add(i)
#         next_guess_list[i] = guessword[i]

#     if (guessword[i] != targetword[i] and  guessword[i] in targetword): # letter is correct but position is incorrect
#         wrong_pos_dict[guessword[i]].add(i)

#     if guessword[i] not in targetword: # if letter is not relevant at all
#         dont_guess_again_set.add(guessword[i]) # intent: "don't guess this letter at any position again"
#         for pos in letter_positions:
#             wrong_pos_dict[guessword[i]].add(pos) # intent: "don't guess this letter at any position again"

# ##### had to do this to get all_perfect_dict to be able to account for duplicate letters in guessword
# for i, letter in enumerate(next_guess_list):
#     if letter != "_":
#         all_perfect_dict[next_guess_list[i]].add(i)

# # print (next_guess_list)

# ##### if no letters are correct, add them all to dont_guess_again_set and pick the best new word from the list that doesn't include any of them (just picks the first)
# if len(set(guessword).difference(set(targetword))) == 5:

#     print ("\n----- No letters were correct. A new word with new letters will be chosen. -----\n")

#     # for letter in guessword:
#     #     for pos in letter_positions:
#     #         dont_guess_again_set[guessword[i]].add(pos)

#     new_word_letters = set()

#     while len(new_word_letters) < wordlength:

#         for letter, freq in sorted_letters_counts:

#             if letter not in dont_guess_again_set:
#                 new_word_letters.add(letter)

#                 if len(new_word_letters) == wordlength:
#                     break
                    
#         for word in all_words:

#             if set(word) == new_word_letters:
#                 guessword = word # setting new guessword to this -- here, it should loop back up to the start of the whole evaluation process
                
#                 # print (f"\nNext guess: {guessword}\n")

#                 break

# ########################################## THIS WORKS ##########################################

# ##### if at least one letter is correct (even in the wrong position)
# else:
#     if simple_solve == False:
#         print ("\n----- At least one letter was correct. A new word with similar letters will be chosen. -----\n")

#         #### getting set of letters previously in incorrect positions
#         try_again_letters = set()
#         for letter, positions in wrong_pos_dict.items():
#             if len(wrong_pos_dict[letter]) > 0 and len(wrong_pos_dict[letter]) != 5: # gets only letters that were guessed and were in wrong positions
#                 if letter not in next_guess_list:
#                     try_again_letters.add(letter)

#         #### fills next guess with try_again_letters in new positions, if they are not already in the word (maximizing letter guessing potential)
#         for i, letter in enumerate(next_guess_list): # i is position, letter is letter in that position
#             if letter == "_":
#                 for lett in try_again_letters:
#                     if lett not in next_guess_list:
#                         next_guess_list[i] = lett

#         if "_" in next_guess_list:
#             print ("YASS BITCHES")
#             valid_word = False
#             while valid_word == False:
            
#                 for i, letter in enumerate(next_guess_list): # i is position, letter is letter in that position
                    
#                     if letter == "_":
                        
#                         for lett, freq in sorted_letters_counts:
                            
#                             if lett not in (next_guess_list and dont_guess_again_set):
                                
#                                 next_guess_list[i] = lett
#                                 # print (next_guess_list)

#                                 if "".join(next_guess_list) in all_words:
#                                     break
#                                 else:
#                                     dont_guess_again_set.add(lett)
#                 break

#             #### janky af and should definitely change this but it works for now
#             dont_guess_again_set = dont_guess_again_set.difference(set(all_perfect_dict))
#             for letter, positions in wrong_pos_dict.items():
#                 if len(positions) == 5:
#                     dont_guess_again_set.add(letter)

#             guessword = "".join(next_guess_list)
#             # print (guessword)

#     elif simple_solve == True:
        
#         valid_letters = set()

#         for letter, positions in all_perfect_dict.items():
#             if len(positions) > 0:
#                 # print (letter)
#                 valid_letters.add(letter)
        
#         for letter, positions in wrong_pos_dict.items():
#             if 0 < len(positions) < 5:
#                 # print (letter)
#                 valid_letters.add(letter)

#         print (dont_guess_again_set)


#         # print (valid_letters)
#         # else:
#         #     for word in all_words:
#         #         if set(word) == valid_letters:
#         #             valid_words.add(word)
        



# # print (valid_letters)
# # print(f"\nAll perfect: {all_perfect_dict}")
# # print(f"Wrong positions: {wrong_pos_dict}")
# # print(f"All wrong: {dont_guess_again_set}")
# # print(next_guess_list)
# print (f"\n{guessword}")

In [51]:
# #### VERSION 5

# #### Step 1
# guess = "aeros"
# target = "tired"

# print (f"Guess: {guess}")
# print (f"Target: {target}\n")

# wordlen = len(guess)
# letter_positions = set(i for i in range(0, wordlen))

# guess_set = set()
# target_set = set() # might not need this for anything -- is cheating

# perfect_dict = {}
# wrong_pos_dict = {}
# dont_guess_again = set()
# guessed_words = set() # running set of guessed words

# #### Step 2 -- ALL PERFECT
# for i in letter_positions: # number of letters in each word (current word and target word)
#     guess_set.add(guess[i])
#     perfect_dict[guess[i]] = set()
#     wrong_pos_dict[guess[i]] = set()
    
#     if guess[i] == target[i]: # letter == correct and position == correct
#         perfect_dict[guess[i]].add(i)

#     if (guess[i] != target[i] and  guess[i] in target): # letter == correct and position != correct
#         wrong_pos_dict[guess[i]].add(i)

#     if guess[i] not in target: # if letter is not relevant at all
#         dont_guess_again.add(guess[i])

# guessed_words.add(guess)

# # print(guess_set) # works
# # print(target_set) # works
# print(f"Perfect dict: {perfect_dict}") # works
# print(f"wrong pos dict: {wrong_pos_dict}") # works
# print(f"dont guess again: {dont_guess_again}\n") # works
    
# #### Step 3 -- ALL PERFECT
# next_letters = set()
# for letter, positions in perfect_dict.items():
#     if len(positions) > 0:
#         next_letters.add(letter)

# for letter, positions in wrong_pos_dict.items():
#     if len(positions) > 0:
#         next_letters.add(letter)

# # print (next_letters)

# # #### Step 4 -- ALL PERFECT
# switch = None
# if len(next_letters) != wordlen:
#     switch = False

# while switch == False:

#     for letter, freq in sorted_letters_counts:
#         if letter not in dont_guess_again:
#             next_letters.add(letter) # duplicates don't matter because it's a set

#         if len(next_letters) == wordlen:
#             switch == True # should break while loop
#             break # breaks for loop (stops adding more letters)
#     break

# print (f"Next letters: {next_letters}")

# #### Step 5
# valid_words = []
# for word in all_words:
#     word_set = set()

#     for letter in word:
#         word_set.add(letter)
    
#     if word_set == next_letters:
#         valid_words.append(word)

# print (f"Valid next words: {valid_words}\n")

# ### Step 6
# valid_word_dicts = []
# for word in valid_words:
#     new_perfect_dict = {}

#     for i in letter_positions:
#         new_perfect_dict[word[i]] = set()

#         if word[i] == target[i]: # letter == correct and position == correct
#             new_perfect_dict[word[i]].add(i)

#     valid_word_dicts.append((word, new_perfect_dict))

# # print(valid_word_dicts)

# #### List of tuples of correct letter positions in new valid words. Eg: [('e', 2), ('a', 3)]
# perfect_letters = []
# for letter, positions in perfect_dict.items():
#     for pos in positions:
#         if len(positions) > 0:
#             perfect_letters.append((letter, pos))

# #### List of tuples of perfect letters. Eg: [('e', 2), ('a', 3)]
# valid_words_perfect_letters_list = []
# for word, dictionary in valid_word_dicts:
#     potential_letters = []
#     for letter, positions in dictionary.items():
#         for pos in positions:
#             if len(positions) > 0:
#                 potential_letters.append((letter, pos))
#     valid_words_perfect_letters_list.append((word, potential_letters))

# print (f"perfect_letters: {perfect_letters}")
# print (f"valid_words_perfect_letters_list: {valid_words_perfect_letters_list}")

# for old_lett, old_pos in perfect_letters:
#     for word, each_list in valid_words_perfect_letters_list:
#         for new_lett, new_pos in each_list:
#             if (old_lett, old_pos) in each_list:
#                 guess = word
#                 guessed_words.add(guess)
#                 break
#             else:                
#                 random_int = random.randint(0, len(valid_words) - 1)
#                 guess = valid_words[random_int]
#                 guessed_words.add(guess)
#             break
#         break
#     break

# guessed_words

In [52]:
# #### VERSION 6 --- THIS WORKS --- FOR SAFEKEEPING

# #### Step 1
# guess = "aeros"
# target = "tired"

# print (f"Guess: {guess}")
# print (f"Target: {target}\n")

# wordlen = len(guess)
# letter_positions = set(i for i in range(0, wordlen))

# guess_set = set()
# target_set = set() # might not need this for anything -- is cheating

# perfect_dict = {}
# wrong_pos_dict = {}
# wrong_pos_set = set()
# dont_guess_again = set()
# guessed_words = [] # running set of guessed words

# guess_num = 0

# ################################################ WHILE LOOP GOES HERE ################################################

# #### Step 2 -- ALL PERFECT
# for i in letter_positions: # number of letters in each word (current word and target word)
#     guess_set.add(guess[i])
#     perfect_dict[guess[i]] = set()
#     wrong_pos_dict[guess[i]] = set()
    
#     if guess[i] == target[i]: # letter == correct and position == correct
#         perfect_dict[guess[i]].add(i)

#     if (guess[i] != target[i] and  guess[i] in target): # letter == correct and position != correct
#         wrong_pos_dict[guess[i]].add(i)
#         wrong_pos_set.add(guess[i])

#     if guess[i] not in target: # if letter is not relevant at all
#         dont_guess_again.add(guess[i])

# guessed_words.append(guess)

# # print(guess_set) # works
# # # print(target_set) # works
# # print(f"Perfect dict: {perfect_dict}") # works
# # print(f"wrong pos dict: {wrong_pos_dict}") # works

    
# #### Step 3 -- ALL PERFECT
# next_letters = set()
# for letter, positions in perfect_dict.items():
#     if len(positions) > 0:
#         next_letters.add(letter)

# for letter, positions in wrong_pos_dict.items():
#     if len(positions) > 0:
#         next_letters.add(letter)

# print (f"next letters: {next_letters}")

# #### List of tuples of correct letter positions in new valid words. Eg: [('e', 2), ('a', 3)]
# perfect_letters = []
# for letter, positions in perfect_dict.items():
#     for pos in positions:
#         if len(positions) > 0:
#             perfect_letters.append((letter, pos))

# #### all words that have correct letters in same spots
# words_matching_correct_letters = []
# for word in all_words:
#     word_set = set()
#     for letter, pos in perfect_letters:
#         if word[pos] == letter:
#             words_matching_correct_letters.append(word)

# #### excluding words with letters in known incorrect positions
# incorrect_positions = []
# for letter, positions in wrong_pos_dict.items():
#     for pos in positions:
#         if len(positions) > 0:
#             incorrect_positions.append((letter, pos))


# #### all words that have correct letters in incorrect spots -- so they can be excluded efficiently
# words_with_incorrect_letters = []
# for word in all_words:
#     word_set = set()
#     for letter, pos in incorrect_positions:
#         if word[pos] == letter:
#             words_with_incorrect_letters.append(word)

# print(f"dont guess again: {dont_guess_again}") # works
# print(f"correct_positions: {perfect_letters}")
# print(f"incorrect_positions: {incorrect_positions}\n")


# # Returns True
# # print(A.issubset(B)) # "if everything in A is in B", returns Bool

# #### all unguessed words that are potential next guesses. Contains all correct letters,
# #### including matching letters in correct positions, doesn't include incorrect letters
# potential_next_guess = []
# for word in words_matching_correct_letters:
#     word_set = set()
#     if word not in words_with_incorrect_letters:
#         for letter in word:
#             word_set.add(letter)
#         if next_letters.issubset(word_set):# == True and word_set.issubset(dont_guess_again) == False:
#             if word not in guessed_words:
#                 potential_next_guess.append(word)
                
# print(f"Number of valid next guesses: {len(potential_next_guess)}")
# # potential_next_guess[:10]

# #### Guessing next word
# randomint = random.randint(0, len(potential_next_guess) - 1) # len will be 1 larger than last index because len counts first as 1 and index counts first as 0
# next_guess = potential_next_guess[randomint]
# guessed_words.append(next_guess)

# ################################################ WHILE LOOP ENDS HERE ################################################

# print(guessed_words)
# print(next_guess)

In [53]:
# #### VERSION 6 -- THIS WORKS

# #### FUNCTION ARGUMENTS -- COPY FROM OTHER FUNCTION
# guess = "aeros"
# target = "tired"
# max_attempts = 10
# verbose = True
# drama = None

# #### FUNCTION
# print (f"Target: {target}\n")

# wordlen = len(guess)
# letter_positions = set(i for i in range(0, wordlen))

# guess_set = set()
# perfect_dict = {}
# wrong_pos_dict = {}
# wrong_pos_set = set()

# dont_guess_again = set()

# guessed_words = [] # running set of guessed words
# guess_num = 0 # baseline for variable
# dont_guess_words = []
# set_of_dont_guess_words = set()
# set_of_incorrect_positions = set()
# incorrect_positions = []


# if max_attempts:
#         max_guesses = max_attempts
# else:
#     max_guesses = len(guess)
# ################################################ WHILE LOOP GOES HERE ################################################

# while guess: # while there is any guess -- there are conditions to break it at the bottom

#     guessed_words.append(guess)

#     guess_num += 1 # each time the guess is processed
#     print(f"Guess {guess_num}: {guess}")

#     guess_set = set()
#     # perfect_dict = {}
#     # wrong_pos_dict = {}
#     wrong_pos_set = set()

#     # dont_guess_again = set()

#     #### Step 2 -- ALL PERFECT
#     for i in letter_positions: # number of letters in each word (current word and target word)
#         guess_set.add(guess[i])

#         if guess[i] not in perfect_dict:
#             perfect_dict[guess[i]] = set()
#         if guess[i] not in wrong_pos_dict:
#             wrong_pos_dict[guess[i]] = set()

#         ### EVALUATE CURRENT GUESS
#         if guess[i] == target[i]: # letter == correct and position == correct
#             perfect_dict[guess[i]].add(i)

#         if (guess[i] != target[i] and  guess[i] in target): # letter == correct and position != correct
#             wrong_pos_dict[guess[i]].add(i)
#             wrong_pos_set.add(guess[i])

#         if guess[i] not in target: # if letter is not relevant at all
#             dont_guess_again.add(guess[i])

#     print (f"perfect dict: {perfect_dict}")

#     #### Step 3 -- ALL PERFECT
#     next_letters = set()
#     for letter, positions in perfect_dict.items():
#         if len(positions) > 0:
#             next_letters.add(letter)

#     for letter, positions in wrong_pos_dict.items():
#         if len(positions) > 0:
#             next_letters.add(letter)

#     # print (f"next letters: {next_letters}")

#     #### List of tuples of correct letter positions in new valid words. Eg: [('e', 2), ('a', 3)]
#     perfect_letters = []
#     for letter, positions in perfect_dict.items():
#         for pos in positions:
#             if len(positions) > 0:
#                 perfect_letters.append((letter, pos))

#     #### all words that have correct letters in same spots
#     words_matching_correct_all = []
#     for word in all_words:
#         word_set = set()
#         for letter, pos in perfect_letters:
#             if word[pos] == letter:
#                 words_matching_correct_all.append(word)

#     print(f'number of potential next guesses TEST: {len(words_matching_correct_all)}')

#     #### excluding words with letters in known incorrect positions
#     # incorrect_positions = []
#     for letter, positions in wrong_pos_dict.items():
#         for pos in positions:
#             if len(positions) > 0:
#                 if (letter, pos) not in incorrect_positions:
#                     incorrect_positions.append((letter, pos))

#     #### all words that have correct letters in incorrect spots -- so they can be excluded efficiently
#     # dont_guess_words = []
#     for word in all_words:
#         word_set = set()
#         for letter, pos in incorrect_positions:
#         # for letter, pos in set_of_incorrect_positions:
#             if word[pos] == letter:
#                 dont_guess_words.append(word)

#     if verbose == True:
#         print(f"correct_positions: {perfect_letters}")
#         print (f"next letters: {next_letters}")
#         print(f"incorrect_positions: {incorrect_positions}")
#         # print(f"incorrect_positions: {set_of_incorrect_positions}")
#         print(f"dont guess again: {dont_guess_again}\n") # works

#     # Returns True
#     # print(A.issubset(B)) # "if everything in A is in B", returns Bool

#     #### all unguessed words that are potential next guesses. Contains all correct letters,
#     #### including matching letters in correct positions, doesn't include incorrect letters
#     potential_next_guesses = []
#     for word in words_matching_correct_all:
#         word_set = set()
#         if word not in dont_guess_words:
#             for letter in word:
#                 word_set.add(letter)

#             if next_letters.issubset(word_set):# == True and word_set.issubset(dont_guess_again) == False:
#                 if word not in guessed_words:
#                     switch = True
#                     for bad_letter in dont_guess_again:
#                         if bad_letter not in word:
#                             potential_next_guesses.append(word)

#     if verbose == True:                    
#         print(f"Number of valid next guesses: {len(potential_next_guesses)}")
#         print (f"Words guessed so far: {guessed_words}")

#     #### Guessing next word
#     if len(potential_next_guesses) == 1:
#         guess = potential_next_guesses[0]
#     else:
#         # randomly guessing next word
#         # randomint = random.randint(1, len(potential_next_guesses) - 1) # len will be 1 larger than last index because len counts first as 1 and index counts first as 0
#         # guess = potential_next_guesses[randomint]
#         for word in potential_next_guesses:
#             for letter, freq in sorted_letters_counts:
#                 if letter not in (next_letters):
#                     if letter in word:
#                         guess = word
#                         break
#                     break

#     if guess == target:
#         if guess_num == 1:
#             print(f"Congratulations! The Wordle has been solved in {guess_num} guess!")
#         else:
#             print(f"\nCongratulations! The Wordle has been solved in {guess_num} guesses!")
#             # print(f"There were still {max_attempts - guess_num} guesses remaining.")
#             # print(f"Here are some statistics about this Wordle:") # come up with some stats about it
#             # print("\n-----------------------------")
#         break

#     if guess_num == max_guesses:
#         if verbose == True:
#             # print("-----------------------------\n")
#             print(f"Unfortunately, the Wordle could not be solved in {max_attempts} guesses.\n")
#             print(f"The target word was '{target}'. Better luck next time!\n")
#             print("-----------------------------\n")
#         else:
#             print(f"\nUnfortunately, the Wordle could not be solved in {max_attempts} guesses.")
#             print(f"The target word was '{target}'. Better luck next time!\n")
#         break
#     else: # if not at max guesses yet
#         if verbose == True:
#             print (f"Next guess: {guess}")
#             print("\n-----------------------------\n")


#     ################################################ WHILE LOOP ENDS HERE ################################################

### `solve_wordle()` function THIS WORKS

In [145]:
#### VERSION 6 -- FUNCTION

def solve_wordle(word_list: list, max_guesses: None, guess: str = None, target: str = None, random_guess: bool = False, random_target: bool = False, verbose: bool = False, drama: float = None, return_stats: bool = False):
    """
    Mimicking the popular web game, this function matches a current word to a target word in the same way a human would, in the most statistically optimal way possible.

    ------
    Parameters:
    ------
    `word_list`: list
        list of valid words to be considered
    `opening_word`: str
        a string -- must be the same length as `target_word`
    `target_word`: str
        a string -- must be the same length as `opening_word`
    `max_guesses`: int
        the maximum number of attempts allowed to solve the Wordle
    `random_guess`: bool
        if True, randomly chooses a starting word from all words within `word_list`. If False, passed starting word must be used instead
    `random_target`: bool
        if True, randomly chooses a target word from all words within `word_list`. If False, passed target word must be used instead
    `verbose`: bool
        if True, prints progress and explanation of how function solves the puzzle. If False, prints only the guessed word at each guess.
    `drama`: float or int
        if int provided, each guess' output is delayed by that number of seconds, else each output is shown as quickly as possible. For ~dRaMaTiC eFfEcT~
    `return_stats`: bool
        if True, prints nothing and returns a dictionary various statistics about the function's performance trying to solve the puzzle

    ------
    Returns:
    ------
    `stats_dict`: dict
        dictionary containing various statistics about the function's performance trying to solve the puzzle
    """

    stats_dict = {}
    stats_dict['first_guess'] = guess

    english_alphabet = "abcdefghijklmnopqrstuvwxyz"

    word_list_sorted_counts = get_letter_counts(english_alphabet, word_list, sort = "descending")

    if random_guess == True:
        randomint_guess = random.randint(0, len(word_list) - 1)
        guess = word_list[randomint_guess]

    if random_target == True:
        randomint_target = random.randint(0, len(word_list) - 1)
        target = word_list[randomint_target]

    wordlen = len(guess)
    letter_positions = set(i for i in range(0, wordlen))

    guess_set = set()
    perfect_dict = {}
    wrong_pos_dict = {}
    wrong_pos_set = set()
    dont_guess_again = set()

    guessed_words = [] # running set of guessed words
    guess_num = 0 # baseline for variable
    dont_guess_words = set()
    incorrect_positions = []

    if max_guesses == None: # if no value is passed, default is len(guess)
        max_guesses = wordlen
    else: # else it is the value passed
        max_guesses = max_guesses


    while guess: # while there is any guess -- there are conditions to break it at the bottom

        guess_num += 1

        guessed_words.append(guess)

        if drama:
            time.sleep(drama)

        # guess_num += 1 # each time the guess is processed
        if return_stats == False:
            if guess_num == 1:
                print("-----------------------------\n")
    
        if return_stats == False:
            print(f"Guess {guess_num}: '{guess}'")

        if guess == target:
            if return_stats == False:
                if guess_num == 1:
                    print(f"Congratulations! The Wordle has been solved in {guess_num} guess, that's amazingly lucky!")
                    print(f"The target word was {target}")
                    # break
            break

        guess_set = set()
        wrong_pos_set = set()

        #### Step 2 -- ALL PERFECT
        for i in letter_positions: # number of letters in each word (current word and target word)
            guess_set.add(guess[i])

            if guess[i] not in perfect_dict:
                perfect_dict[guess[i]] = set()
            if guess[i] not in wrong_pos_dict:
                wrong_pos_dict[guess[i]] = set()

            ### EVALUATE CURRENT GUESS
            if guess[i] == target[i]: # letter == correct and position == correct
                perfect_dict[guess[i]].add(i)

            if (guess[i] != target[i] and  guess[i] in target): # letter == correct and position != correct
                wrong_pos_dict[guess[i]].add(i)
                wrong_pos_set.add(guess[i])

            if guess[i] not in target: # if letter is not relevant at all
                dont_guess_again.add(guess[i])

        #### Step 3 -- ALL PERFECT
        next_letters = set()
        for letter, positions in perfect_dict.items():
            if len(positions) > 0:
                next_letters.add(letter)

        for letter, positions in wrong_pos_dict.items():
            if len(positions) > 0:
                next_letters.add(letter)

        #### List of tuples of correct letter positions in new valid words. Eg: [('e', 2), ('a', 3)]
        perfect_letters = []
        for letter, positions in perfect_dict.items():
            for pos in positions:
                if len(positions) > 0:
                    perfect_letters.append((letter, pos))

        #### all words that have correct letters in same spots
        words_matching_correct_all = []
        for word in word_list:
            word_set = set()
            for letter, pos in perfect_letters:
                if word[pos] == letter:
                    words_matching_correct_all.append(word)

        #### excluding words with letters in known incorrect positions
        for letter, positions in wrong_pos_dict.items():
            for pos in positions:
                if len(positions) > 0:
                    if (letter, pos) not in incorrect_positions:
                        incorrect_positions.append((letter, pos))

        #### all words that have correct letters in incorrect spots -- so they can be excluded efficiently
        for word in word_list:
            word_set = set()
            for letter, pos in incorrect_positions:
                if word[pos] == letter:
                    dont_guess_words.add(word)

        for bad_letter in dont_guess_again:
            for word in word_list:
                if (bad_letter in word and word not in dont_guess_words):
                    dont_guess_words.add(word)

        if return_stats == False:
            if verbose == True:
                print(f"Letters in correct positions:\n\t{perfect_letters}\n")
                print (f"Letters to guess again:\n\t{next_letters}\n")
                print(f"Letters in incorrect positions:\n\t{incorrect_positions}\n")
                print(f"Letters to not guess again:\n\t{dont_guess_again}\n") # works

        # Returns True
        # print(A.issubset(B)) # "if everything in A is in B", returns Bool

        #### all unguessed words that are potential next guesses. Contains all correct letters,
        #### including matching letters in correct positions, doesn't include incorrect letters
    
        potential_next_guesses = set()

        if len(perfect_letters) == 0 and len(incorrect_positions) == 0: # if there are NEITHER perfect letters, NOR incorrect positions, ....
            for word in word_list:
                if word not in dont_guess_words:
                    if word not in guessed_words:
                        potential_next_guesses.add(word)
                                        
            # print(f"GUESS {guess_num} : TEST 1")

        if len(perfect_letters) == 0 and len(incorrect_positions) != 0: # if there are no perfect letters whatsoever, but there ARE incorrect positions ....
            for word in word_list:
                for incor_letter, incor_pos in incorrect_positions:
                    if word[incor_pos] != incor_letter:
                        if word not in dont_guess_words: # just in case
                            word_set = set()
                            for letter in word:
                                word_set.add(letter)
                                if next_letters.issubset(word_set):
                                    if word not in guessed_words:
                                        for bad_letter in dont_guess_again:
                                            if bad_letter not in word:
                                                # potential_next_guesses.append(word)
                                                potential_next_guesses.add(word)
            
            # print(f"GUESS {guess_num} : TEST 2")

        else:
            for word in word_list:
                if word not in dont_guess_words: # just in case
                    word_set = set()
                    for letter in word:
                        word_set.add(letter)
                        if next_letters.issubset(word_set):
                            if word not in guessed_words:
                                for bad_letter in dont_guess_again:
                                    if bad_letter not in word:
                                        # potential_next_guesses.append(word)
                                        potential_next_guesses.add(word)
                                        
            # print(f"GUESS {guess_num} : TEST 3")

        if return_stats == False:
            if verbose == True:
                print(f"At this point:")
                print(f"\t{len(word_list) - len(potential_next_guesses)}, {round((len(word_list) - len(potential_next_guesses)) / len(word_list) * 100, 2)}% of total words have been eliminated, and")
                print(f"\t{len(potential_next_guesses)}, {round(len(potential_next_guesses) / len(word_list) * 100, 2)}% of total words remain possible.\n")
                
                if len(potential_next_guesses) <= 20:
                    print(f"Potential next guesses:\n\t{potential_next_guesses}\n")
                print (f"Words guessed so far:\n\t{guessed_words}.\n")

        #### Guessing next word
        if len(potential_next_guesses) == 1:
            guess = list(potential_next_guesses)[0]
        else:
            best_next_guesses = set()
            for word in potential_next_guesses:
                for letter, freq in word_list_sorted_counts:
                    if letter not in dont_guess_again:
                        if len(next_letters) > 0:
                            if letter in next_letters:
                                if letter in word:
                                    best_next_guesses.add(word)
                                    break
                        else:
                            if letter in word:
                                best_next_guesses.add(word)
                                break

            guess = list(best_next_guesses)[0]

        if guess_num == max_guesses: # if at max guesses allowed
            if return_stats == False:
                if verbose == True:
                    # print("-----------------------------\n")
                    print(f"Unfortunately, the Wordle could not be solved in {max_guesses} guesses.\n")
                    print(f"The target word was '{target}'. Better luck next time!\n")
                    print("-----------------------------\n")
                else:
                    print(f"\nUnfortunately, the Wordle could not be solved in {max_guesses} guesses.")
                    print(f"The target word was '{target}'. Better luck next time!\n")
            break
        else: # if not at max guesses yet allowed
            if return_stats == False:
                if verbose == True:
                    print (f"Next guess:\n\t'{guess}'")
                    print("\n-----------------------------\n")

        if guess == target:
            guess_num += 1
            if return_stats == False:
                print(f"Guess {guess_num}: '{guess}'\n")
                print(f"Congratulations! The Wordle has been solved in {guess_num} guesses!")
                if max_guesses - guess_num == 0:
                    print(f"Lucky! It was the last guess.")
                else:
                    print(f"There were still {max_guesses - guess_num} guesses remaining.")
            if return_stats == False:                    
                print(f"\nThe target word was '{target}'.")
                print("\n-----------------------------")
            break

        stats_dict['num_guesses'] = guess_num
        stats_dict['target_word'] = target



In [146]:
# guess_word = "feeet"
# guess_word = "works"
# guess_word = "creed" # hard combo 1
# target_word = "alloy" # hard combo 1

solve_wordle(word_list = official_words, max_guesses = 10, guess = "toast", target = "creed", random_guess = False, random_target = False, verbose = False, drama = 0, return_stats = False)

-----------------------------

Guess 1: 'toast'
Guess 2: 'bible'
Guess 3: 'mercy'
Guess 4: 'creek'
Guess 5: 'creep'
Guess 6: 'cheer'
Guess 7: 'creed'

Congratulations! The Wordle has been solved in 7 guesses!
There were still 3 guesses remaining.

The target word was 'creed'.

-----------------------------


In [56]:
# # randomly generate starting word
# randomint_guess = random.randint(0, len(best_opening_words_list) - 1) # len will be 1 larger than last index because len counts first as 1 and index counts first as 0
# randomint_target = random.randint(0, len(all_words) - 1) # len will be 1 larger than last index because len counts first as 1 and index counts first as 0